# Simulating EPA-RE using points of low-order

In [1]:
import pickle
import itertools

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from collections import Counter

from pathlib import Path
from random import randint
from typing import Type, Any

from bs4 import BeautifulSoup
from tqdm.auto import tqdm, trange

from pyecsca.ec.params import DomainParameters, get_params
from pyecsca.ec.mult import *
from pyecsca.sca.re.rpa import MultipleContext, rpa_distinguish, RPA, multiples_computed
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.misc.utils import TaskExecutor

from common import MultIdent, MultResults, enable_spawn, spawn_context

## Initialize

In [2]:
# All dbl-and-add multipliers from https://github.com/J08nY/pyecsca/blob/master/pyecsca/ec/mult

window_mults = [
    MultIdent(SlidingWindowMultiplier, width=3),
    MultIdent(SlidingWindowMultiplier, width=4),
    MultIdent(SlidingWindowMultiplier, width=5),
    MultIdent(SlidingWindowMultiplier, width=6),
    MultIdent(FixedWindowLTRMultiplier, m=2**4),
    MultIdent(FixedWindowLTRMultiplier, m=2**5),
    MultIdent(FixedWindowLTRMultiplier, m=2**6),
    MultIdent(WindowBoothMultiplier, width=3),
    MultIdent(WindowBoothMultiplier, width=4),
    MultIdent(WindowBoothMultiplier, width=5),
    MultIdent(WindowBoothMultiplier, width=6)
]
naf_mults = [
    MultIdent(WindowNAFMultiplier, width=3),
    MultIdent(WindowNAFMultiplier, width=4),
    MultIdent(WindowNAFMultiplier, width=5),
    MultIdent(WindowNAFMultiplier, width=6),
    MultIdent(BinaryNAFMultiplier, direction=ProcessingDirection.LTR),
    MultIdent(BinaryNAFMultiplier, direction=ProcessingDirection.RTL)
]
comb_mults = [
    MultIdent(CombMultiplier, width=2),
    MultIdent(CombMultiplier, width=3),
    MultIdent(CombMultiplier, width=4),
    MultIdent(CombMultiplier, width=5),
    MultIdent(CombMultiplier, width=6),
    MultIdent(BGMWMultiplier, width=2, direction=ProcessingDirection.LTR),
    MultIdent(BGMWMultiplier, width=3, direction=ProcessingDirection.LTR),
    MultIdent(BGMWMultiplier, width=4, direction=ProcessingDirection.LTR),
    MultIdent(BGMWMultiplier, width=5, direction=ProcessingDirection.LTR),
    MultIdent(BGMWMultiplier, width=6, direction=ProcessingDirection.LTR),
    MultIdent(BGMWMultiplier, width=2, direction=ProcessingDirection.RTL),
    MultIdent(BGMWMultiplier, width=3, direction=ProcessingDirection.RTL),
    MultIdent(BGMWMultiplier, width=4, direction=ProcessingDirection.RTL),
    MultIdent(BGMWMultiplier, width=5, direction=ProcessingDirection.RTL),
    MultIdent(BGMWMultiplier, width=6, direction=ProcessingDirection.RTL)
]
binary_mults = [
    MultIdent(LTRMultiplier, always=False),
    MultIdent(LTRMultiplier, always=True),
    MultIdent(RTLMultiplier, always=False),
    MultIdent(RTLMultiplier, always=True),
    MultIdent(CoronMultiplier)
]
other_mults = [
    MultIdent(FullPrecompMultiplier, always=False),
    MultIdent(FullPrecompMultiplier, always=True),
    MultIdent(SimpleLadderMultiplier, complete=True),
    MultIdent(SimpleLadderMultiplier, complete=False)
]

all_mults = window_mults + naf_mults + binary_mults + other_mults + comb_mults

In [3]:
print(len(all_mults))

41


In [4]:
# Needs imports on the inside to be spawn enabled to save memory.

def get_general_multiples(bits: int, samples: int = 1000) -> MultResults:
    from random import randint
    results = []
    for _ in range(samples):
        big_scalar = randint(1, 2**bits)
        results.append({big_scalar})
    return MultResults(results, samples)

def get_general_n_multiples(bits: int, n: int, samples: int = 1000) -> MultResults:
    from random import randint
    results = []
    for _ in range(samples):
        smult = set()
        for i in range(n):
            b = randint(1,256)
            smult.add(randint(2**b,2**(b+1)))
        results.append(smult)
    return MultResults(results, samples)

def get_small_scalar_multiples(mult: MultIdent, params: DomainParameters, bits: int, samples: int = 1000, use_init: bool = True, use_multiply: bool = True) -> MultResults:
    from pyecsca.sca.re.rpa import multiples_computed
    from random import randint
    results = []
    for _ in range(samples):
        big_scalar = randint(1, 2**bits)
        results.append(multiples_computed(big_scalar, params, mult.klass, mult.partial, use_init, use_multiply))
    return MultResults(results, samples)

## Prepare

In [5]:
multiples_mults = {}

In [6]:
category = "secg"
curve = "secp256r1"
params = get_params(category, curve, "projective")
num_workers = 20
bits = params.order.bit_length()
samples = 1000
selected_mults = all_mults

## Run
Run this cell as many times as you want. It will accumulate into multiples_mults.

In [7]:
with TaskExecutor(max_workers=num_workers, mp_context=spawn_context) as pool, enable_spawn(get_small_scalar_multiples) as target:
    for mult in selected_mults:
        pool.submit_task(mult,
                         target,
                         mult, params, bits, samples)
    for mult, future in tqdm(pool.as_completed(), desc="Computing small scalar distributions.", total=len(pool.tasks)):
        print(f"Got {mult}.")
        if error := future.exception():
            print(error)
            continue
        res = future.result()
        if mult not in multiples_mults:
            multiples_mults[mult] = res
        else:
            # Accumulate
            multiples_mults[mult].merge(res)

Computing small scalar distributions.:   0%|          | 0/41 [00:00<?, ?it/s]

Got SlidingWindowMultiplier_()_{'width': 4}.


NameError: name 'res' is not defined

### Save

In [ ]:
with open(f"multiples_{category}_{curve}_{bits}","wb") as h:
    pickle.dump(multiples_mults, h)

### Load

In [ ]:
with open(f"multiples_{category}_{curve}_{bits}", "rb") as f:
    multiples_mults = pickle.load(f)